## Step 1: Load Data from Files

### 1.1. Load INX and SIMX Files

In [9]:
def load_inx_file(filepath):
    with open(filepath, 'r') as file:
        data = file.readlines()
    return data

def load_simx_file(filepath):
    with open(filepath, 'r') as file:
        data = file.readlines()
    return data

In [10]:
import os

# One directory up to the project root
base_dir = os.path.dirname(os.getcwd())

# Status quo directory path
statusquo_dir = os.path.join(base_dir, 'data', 'statusquo')

# Relative paths
inx_file = os.path.join(statusquo_dir, 'Playground_statusquo.INX')
simx_file = os.path.join(statusquo_dir, 'Playground_statusquo.simx')

# Load the files
inx_data = load_inx_file(inx_file)
simx_data = load_simx_file(simx_file)

# Displaying a snippet of the data (for debugging)
print("INX File Data:", inx_data[:5])  # first 5 lines
print("SIMX File Data:", simx_data[:5])


INX File Data: ['<ENVI-MET_Datafile>\n', '<Header>\n', '<filetype>INPX ENVI-met Area Input File</filetype>\n', '<version>440</version>\n', '<revisiondate>13.09.2024 19:32:05</revisiondate>\n']
SIMX File Data: ['<ENVI-MET_Datafile>\n', '<Header>\n', '<filetype>SIMX</filetype>\n', '<version>2</version>\n', '<revisiondate>13.09.2024 17:58:04</revisiondate>\n']


### 1.2 Load Climate Input (FOX) Data

In [11]:
# Relative paths the FOX file
fox_file = os.path.join(statusquo_dir, 'Playground.FOX')

# Function to load the FOX file
def load_fox_file(filepath):
    with open(filepath, 'r') as file:
        data = file.readlines()
    return data

# Load FOX data
fox_data = load_fox_file(fox_file)

# Display a snippet of the climate input data
print("FOX File Data:", fox_data[:5])

FOX File Data: ['{\n', '    "fileType": "ENVI-met JSON Forcing File",\n', '    "metaData": {\n', '        "fileDescription": "FULL FORCING FOR ENVI-met",\n', '        "remark": "",\n']


## Step 2: Extract and Visualize Data

### Work with the .nc File

##### 1. Loading the netCDF File:

In [12]:
import xarray as xr
#import os

# Use the current working directory but go one directory up to the project root
base_dir = os.path.dirname(os.getcwd())  # Go up one directory level

# Define the status quo directory path
statusquo_dir = os.path.join(base_dir, 'data', 'statusquo')

# Use relative paths for your .nc file
nc_file = os.path.join(statusquo_dir, 'Playground_2024-07-06_04.00.00.nc')

# Load the netCDF file using xarray
ds = xr.open_dataset(nc_file)

# Explore the contents of the netCDF file
print(ds)


ModuleNotFoundError: No module named 'xarray'